In [ ]:
import os
import torch
from transformers import T5Tokenizer, T5Model
import sentencepiece as spm

# Verificar si CUDA está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

# Cargar el modelo preentrenado y el tokenizador
tokenizer = T5Tokenizer.from_pretrained("vgaraujov/t5-base-spanish")
model = T5Model.from_pretrained("vgaraujov/t5-base-spanish").to(device)

# Función para corregir texto
def correct_text(text):
    input_text = f"corrige: {text}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)
    outputs = model.generate(inputs.input_ids, max_length=512, num_beams=4, early_stopping=True)
    corrected_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return corrected_text

# Directorios de entrada y salida
input_dir = r'..\..\data\output\output_text_test'
output_dir = r'..\..\data\output\output_correct_text'

# Función para procesar un archivo
def process_file(input_file_path, output_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    # Imprimir el texto original
    print(f'Texto original:\n{text}\n')

    # Corregir el texto
    corrected_text = correct_text(text)

    # Imprimir la corrección
    print(f'Texto corregido:\n{corrected_text}\n')

    # Guardar el texto corregido en el archivo de salida
    with open(output_file_path, 'w', encoding='utf-8') as file:
        file.write(corrected_text)

    print(f'Procesado y guardado: {output_file_path}\n')

# Recorrer recursivamente el directorio de entrada
for root, dirs, files in os.walk(input_dir):
    for file in files:
        if file.lower().endswith('.txt'):
            # Obtener la ruta completa del archivo de entrada
            input_file_path = os.path.join(root, file)

            # Crear la ruta de salida correspondiente
            relative_path = os.path.relpath(root, input_dir)
            output_dir_path = os.path.join(output_dir, relative_path)
            os.makedirs(output_dir_path, exist_ok=True)

            # Crear la ruta completa del archivo de salida
            output_file_path = os.path.join(output_dir_path, file)

            # Procesar el archivo
            process_file(input_file_path, output_file_path)
            print(f'Procesado y guardado: {output_file_path}')